In [2]:
import os
from dotenv import load_dotenv
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI 

load_dotenv(dotenv_path='.env')
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
print(OPENAI_API_KEY[:9])

gsk_chbQQ


### 문제 2-1 : 콤마 구분 리스트 파서 활용
- 출력 파서: CommaSeparatedListOutputParser
- 문제
    -  사용자가 관심있는 분야(예: "음식", "스포츠", "영화" 등)를 입력하면, 해당 분야와 관련된 한국의 유명한 장소나 활동 5가지를 콤마로 구분된 리스트로 출력하는 프로그램을 작성하세요.
- 요구사항:
CommaSeparatedListOutputParser 사용
    - 사용자 입력을 받아 동적으로 처리
    - 한국 관련 내용으로 추천


In [ ]:
from langchain_core.output_parsers import CommaSeparatedListOutputParser
from langchain_core.prompts import PromptTemplate
from langchain_openai import ChatOpenAI
import csv

# 콤마로 구분된 리스트 출력 파서 초기화
output_parser = CommaSeparatedListOutputParser()

# 출력 형식 지침 가져오기
format_instructions = output_parser.get_format_instructions()

# 프롬프트 템플릿 설정
prompt = PromptTemplate(
    template="{subject}와 관련된 한국의 유명한 장소나 활동 5가지.\n{format_instructions}",
    input_variables=["subject"],
    partial_variables={"format_instructions": format_instructions},
)

llm = ChatOpenAI(
    api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

chain = prompt | llm | output_parser
result = chain.invoke({"subject": "음식"})

print("한국과 관련된 목록:")
print(result)

한국과 관련된 음식 목록:
['광장시장 먹자골목', '전주 한옥마을 맛집거리', '부산 자갈치시장', '제주도 흑돼지 맛집 투어', '김장문화체험']


### 문제 2-2 : 영화 리뷰 감정 분석기
- 출력 파서: EnumOutputParser
- 문제
    - 영화 리뷰 텍스트를 입력받아 감정을 "긍정", "부정", "보통" 중 하나로 분류하는 시스템을 만드세요.
- 요구사항:
    - EnumOutputParser와 Enum 클래스 사용
    - 여러 개의 테스트 리뷰로 검증
    - 감정 분류 결과를 깔끔하게 출력


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import EnumOutputParser, OutputFixingParser
from langchain.schema import OutputParserException

from enum import Enum
from pprint import pprint

class Review(str, Enum):
    POSITIVE = "긍정"
    NEGATIVE = "부정"
    NORMAL = "보통"


parser = EnumOutputParser(enum=Review)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿
template = """
당신은 영화 리뷰 전문가입니다.
다음 영화리뷰를 보고 반드시 아래 세 가지 중 하나의 단어로만 답변하세요.

텍스트: "{text}"

{format_instructions}

중요 규칙:
1. 반드시 "긍정", "부정", "보통" 중 하나의 단어만 출력하세요
2. 다른 설명이나 부가 설명을 추가하지 마세요
3. 이모지나 특수문자도 포함하지 마세요
4. 오직 하나의 단어만 출력하세요

답변:"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0  # 일관성을 위해 0으로 설정
)

texts = [
    "이 영화 정말 재미없어요. 시간 낭비였습니다.",
    "배우들의 연기가 훌륭하고 스토리도 감동적이었어요!",
    "그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.",
]


# 안전한 감정 분석 함수 (에러 처리 포함)
def safe_revie_analysis(text):
    """안전한 영화 리뷰 함수 - 에러 처리 포함"""
    try:
        # 기본 체인 생성
        chain = prompt | model | parser
        
        # 분석 실행
        result = chain.invoke({"text": text})
        return result, None
        
    except OutputParserException as e:
        return None, f"파싱 오류: {str(e)[:100]}..."
    except Exception as e:
        return None, f"일반 오류: {str(e)[:100]}..."

def run_review_analysis():
    """실제 리뷰 분석"""
    print("리뷰 분석 결과")
    for i,text in enumerate(texts,1):
        print(f"\n{i}. 리뷰 텍스트: {text}")
        result, error = safe_revie_analysis(text)
        print(f"   리뷰: {result.value} ")

# 실제 분석 실행 (API 키가 있는 경우)
try:
    run_review_analysis()
except Exception as e:
    print("API 키가 설정되지 않았거나 네트워크 오류:")
    print("실제 실행을 위해서는 OpenAI API 키를 설정하세요.")
    print(f"오류 상세: {e}")
    

리뷰 분석 결과

1. 텍스트: 이 영화 정말 재미없어요. 시간 낭비였습니다.
   감정: 부정 

2. 텍스트: 배우들의 연기가 훌륭하고 스토리도 감동적이었어요!
   감정: 긍정 

3. 텍스트: 그냥 무난한 영화였습니다. 나쁘지도 좋지도 않아요.
   감정: 보통 


#### 문제 2-3 : 학생 정보 구조화 시스템
- 출력 파서: PydanticOutputParser
- 문제
    - 학생의 자유 형식 자기소개를 입력받아 이름, 나이, 전공, 취미 리스트, 목표를 구조화된 형태로 추출하는 시스템을 만드세요.
- 요구사항:
    - PydanticOutputParser와 BaseModel 사용
    - 각 필드에 적절한 타입과 설명 추가
    - 자유 형식의 텍스트에서 정보 추출


In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import PydanticOutputParser

from pydantic import BaseModel, Field
from typing import List
from pprint import pprint

# 출력 구조를 정의하는 Pydantic 모델
class Introduction(BaseModel):
    name: str = Field(description="학생 이름")
    age: str = Field(description="학생 나이")
    major: str = Field(description="학생 전공")
    hobby: List[str] = Field(description="취미 리스트")
    goal: str = Field(description="학생 목표")

# Pydantic 출력 파서 초기화
parser = PydanticOutputParser(pydantic_object=Introduction)

# 프롬프트 템플릿 설정
template = """
학생의 자유 형식 자기 소개에 따라 이름, 나이, 전공, 취미리스트, 목표를 구조화된 형태로 추출해주세요
요청: {query}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)

# 파서의 지시사항을 프롬프트에 주입
prompt = prompt.partial(
    format_instructions=parser.get_format_instructions()
)

model = ChatOpenAI(
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 체인 구성 및 실행
query =  "안녕하세요! 저는 김민수이고 22살입니다. \
            컴퓨터공학을 전공하고 있어요. \
            취미로는 게임하기, 영화보기, 코딩을 좋아합니다. \
            앞으로 훌륭한 개발자가 되는 것이 목표입니다."
chain = prompt | model | parser
output = chain.invoke({"query": query})

pprint(output)

<class '__main__.Introduction'>
Introduction(name='김민수', age='22', major='컴퓨터공학', hobby=['게임하기', '영화보기', '코딩'], goal='훌륭한 개발자가 되는 것')


#### 문제 2-4 : 여행 계획 분석기
- 출력 파서: StructuredOutputParser
- 문제
    - 여행 후기나 계획 텍스트를 입력받아 여행지, 기간, 예산, 추천도(1-5점), 주요 활동 리스트를 구조화된 형태로 추출하는 시스템을 만드세요.
- 요구사항:
    - StructuredOutputParser와 ResponseSchema 사용
    - 5개의 필드를 정의하여 정보 추출
    - 자연어 텍스트에서 핵심 정보 파싱


In [13]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

from pprint import pprint

response_schemas = [
    ResponseSchema(name="destination", description="여행지"),
    ResponseSchema(name="duration", description="여행기간"),
    ResponseSchema(name="budget", description="여행 예산"),
    ResponseSchema(name="rating", description="5점 만점의 추천도"),
    ResponseSchema(name="activities", description="주요 활동리스트")
]
# 파서 초기화
parser = StructuredOutputParser.from_response_schemas(response_schemas)
format_instructions = parser.get_format_instructions()

# 프롬프트 템플릿
template = """
다음 제품 여행 후기나 계획을 분석하세요. 리뷰 내용: {travel}

{format_instructions}
"""

prompt = ChatPromptTemplate.from_template(template)
prompt = prompt.partial(format_instructions=format_instructions)

model = ChatOpenAI(
    #api_key=OPENAI_API_KEY,
    base_url="https://api.groq.com/openai/v1",  # Groq API 엔드포인트
    model="moonshotai/kimi-k2-instruct-0905",
    temperature=0.7
)

# 테스트 여행 계획/후기 데이터
travel = """
지난 주에 부산으로 2박 3일 여행을 다녀왔어요. 
총 30만원 정도 썼는데 해운대에서 바다구경하고, 자갈치시장에서 회 먹고, 감천문화마을도 구경했어요. 
정말 만족스러운 여행이었습니다. 5점 만점에 4점 정도 줄 수 있을 것 같아요."
"""
# 체인 실행
chain = prompt | model | parser

output = chain.invoke({"travel": travel})

# 결과 출력 (Pretty Print)
print("===== 분석 결과 =====")
pprint(output)

===== 분석 결과 =====
{'activities': '해운대 바다구경, 자갈치시장 회 먹기, 감천문화마을 구경',
 'budget': '30만원',
 'destination': '부산',
 'duration': '2박 3일',
 'rating': '4점'}
